Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. Create a convolutional neural network that trains to 100% accuracy on these images, which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import zipfile

In [2]:
DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs ={}):
    if (logs.get('accuracy') > DESIRED_ACCURACY):
      print('Reached 99.9% accuracy so cancelling training!')
      self.model.stop_training = True

callbacks = myCallback()

--2022-01-23 02:14:23--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.128, 173.194.218.128, 64.233.170.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2022-01-23 02:14:23 (212 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [3]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([keras.layers.Conv2D(64,(3,3),activation = 'relu',input_shape=(150,150,3)),
                                    keras.layers.MaxPooling2D(2,2),
                                    keras.layers.Conv2D(32,(3,3),activation = 'relu'),
                                    keras.layers.MaxPooling2D(2,2),
                                    keras.layers.Conv2D(32,(3,3),activation = 'relu'),
                                    keras.layers.MaxPooling2D(2,2),
                                    keras.layers.Flatten(),
                                    keras.layers.Dense(128, activation= 'relu'),
                                    keras.layers.Dense(1, activation= 'sigmoid')
 
# Your Code Here
])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer = keras.optimizers.RMSprop(0.001), loss= keras.losses.binary_crossentropy, metrics = ['accuracy'])

In [4]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(1/255)

train_generator = train_datagen.flow_from_directory( '/tmp/h-or-s', 
                                                    target_size = (150,150),
                                                    batch_size=10,
                                                    class_mode='binary')
        
# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [5]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(train_generator, steps_per_epoch=8,epochs=15, verbose =1, callbacks= [callbacks])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/15
8/8 [==============================] - 5s 482ms/step - loss: 172.0724 - accuracy: 0.5250
Epoch 2/15
8/8 [==============================] - 4s 463ms/step - loss: 1.4351 - accuracy: 0.7875
Epoch 3/15
8/8 [==============================] - 4s 472ms/step - loss: 1.8264 - accuracy: 0.7250
Epoch 4/15
8/8 [==============================] - 4s 473ms/step - loss: 0.1424 - accuracy: 0.9500
Epoch 5/15
8/8 [==============================] - 4s 470ms/step - loss: 0.0062 - accuracy: 1.0000
